In [ ]:
MAX_IMAGE_NUM = 100000

# -------------------------------------
# calculate IOU of two rects
#     input: r1, r2 ~ (x, y, w, h)
#     output: iou
def calc_iou(r1, r2):
    ox1 = max(r1[0], r2[0])
    oy1 = max(r1[1], r2[1])
    ox2 = min(r1[0]+r1[2], r2[0]+r2[2])
    oy2 = min(r1[1]+r1[3], r2[1]+r2[3])
    oa = (ox2 - ox1)*(oy2 - oy1)
    return oa*1.0/(r1[2]*r1[3]+r2[2]*r2[3] - oa)

# -------------------------------------
# group the detection results, according to their image_id
# image_ids is assumed smaller than MAX_IMAGE_NUM
#     input: list of objects, each object is a dict contains 'image_id', 'category_id', 'bbox', 'score'
#     output: list of list of objects, number of sublists is equal to number of images (empty sublist may exist)
def group_bboxs(obj_list):    
    im_num = 0
    im_bboxs = [list() for i in range(MAX_IMAGE_NUM)]
    for obj in obj_list:
        im_bboxs[obj['image_id']].append(obj)
        im_num = max(im_num, obj['image_id'])
    return im_bboxs[:im_num + 1]

# -------------------------------------
# judge a list, number of images, is correct or not, with score
#     input: list of list of objects, retruned by group_bboxs
#     output: judge results, each element is a pair (0/1, score). 0/1 means if the image is kongbao or not.
def judge_detection_results(dets, gt=None, iou=0.5):
    is_kongbao = (gt is None)
    
    res = []    
    for im_idx, im_det in enumerate(dets):
        if is_kongbao:            
            max_score = 0
            for obj in im_det:
                max_score = max(max_score, obj['score'])
            judge = (0, max_score)            
        else:
            # normal image, find the max_score for corrected ones.            
            # How to determine an image is correct or not
            
            # 1. Any contraband object is detected                        
#            for obj1 in im_det:
#                max_score = max(max_score, obj1['score'])                        
            
            # 2. At least one object is correctly detected
#             max_score = 0
#             for obj1 in im_det:
#                 for obj2 in gt[im_idx]:
#                     if obj1['category_id'] == obj2['category_id'] and calc_iou(obj1['bbox'], obj2['bbox']) >= iou:
#                         max_score = max(max_score, obj1['score'])                        
#             judge = (1, max_score)
            # 3. judge each instance
            max_score = 0
            for obj1 in im_det:
                for obj2 in gt[im_idx]:
                    if obj1['category_id'] == obj2['category_id'] and calc_iou(obj1['bbox'], obj2['bbox']) >= iou:
                        max_score = max(max_score, obj1['score'])                        
                # no matched ground truth
                if max_score == 0:
                    judge = (0, max_score)
                else:
                    judge = (obj1['category_id'], max_score)
    
            
        res.append(judge)    
    return res

# -------------------------------------
# generate roc curve for overall and each category
#     input: judged_detection results, 
#     output: list of (hit rate, false rate, threshold)
def generate_rocs(dets, categories):
    dets.sort(key=lambda x: -x[1])
    
    counts = [0]*(len(categories)+1)
    for item in dets:
        counts[item[0]] += 1
        
    rocs = [list() for c in range(len(categories)+1)]
    hits = [0]*(len(categories)+1)
    fg_hits = 0
    fg_num = sum(counts[1:])
    for x in dets:
        hits[x[0]] += 1
        if x[0] != 0:
            fg_hits += 1
        rocs[0].append((fg_hits*1.0/fg_num, hits[0]*1.0/counts[0], x[1]))
        rocs[x[0]].append((hist[x[0]]*1.0/counts[x[0]], hits[0]*1.0/counts[0], x[1]))
    return rocs


def generate_roc(wjp, kb):
    # the first version
    # 1. A WJP image is correct, if and only if one of its det is correct
    # 2. A KongBao image is correct, if and only if none of its det is correct
    # 
    # We calculate roc as follows:
    # 1) keep only the max score det for each image, so once thres is setup, best results of KongBao and WJP is obtained.
    # 2) sort WJP and KongBao together, WJP is 1, KongBao is 0
    # 3) detection rate is given by how much percentage of WJP images are detected, 
    #    and false detection rate is given by how much percentage of KongBao images are detected,
    #    judged by aboving a given threshold, which will decrease by iterating the sorted scores from 1 to 0.
    wjp_num = len(wjp)
    kb_num = len(kb)
    print 'In total %d WJP, and %d KB' % (wjp_num, kb_num)
    dets = wjp + kb
    dets.sort(key=lambda x: -x[1])
    wjp_hits = 0
    kb_hits = 0
    roc = []
    for x in dets:
        if x[0] == 0:
            kb_hits += 1
        else:
            wjp_hits += 1
        roc.append((wjp_hits*1.0/wjp_num, kb_hits*1.0/kb_num, x[1]))
    return roc

# -------------------------------------
#
def get_threshold_by_false_rate(roc, fr):
    for (idx, item) in enumerate(roc):
        if item[1] > fr:
            return roc[idx-1]
    return roc[-1]

# -------------------------------------    
def get_threshold_by_hit_rate(roc, hr):
    for (idx, item) in enumerate(roc):
        if item[0] > hr:
            return roc[idx]
    return roc[-1]    


In [ ]:
import json
import os
import sys
from eval_funs import *


def load_json_file(json_file):
    with open(json_file, 'rt') as fp:
        return json.load(fp)

def get_res_folder(model):
    model_map = {
         'sr1': 'security_r1_e2e_faster_rcnn_R-101-FPN_1x.yaml',
         's1': 'security1_e2e_faster_rcnn_R-101-FPN_1x.yaml',
         's1a': 'security1a_e2e_faster_rcnn_R-101-FPN_1x.yaml',         
         's2': 'security2_e2e_faster_rcnn_R-101-FPN_1x.yaml',         
         's2p': 'security2p_e2e_faster_rcnn_R-101-FPN_1x.yaml',             
         's3': 'security3_e2e_faster_rcnn_R-101-FPN_1x.yaml',
         's3_rpnfg0.4': 'security3_e2e_faster_rcnn_R-101-FPN_rpnfg0.4_1x.yaml',
         's3_rpnfg0.3': 'security3_e2e_faster_rcnn_R-101-FPN_rpnfg0.3_1x.yaml',            
         's3_rpnfg0.2': 'security3_e2e_faster_rcnn_R-101-FPN_rpnfg0.2_1x.yaml',                                          
         's3_fg0.05': 'security3_e2e_faster_rcnn_R-101-FPN_fg0.05_1x.yaml',   
         's3_fg0.005': 'security3_e2e_faster_rcnn_R-101-FPN_fg0.005_1x.yaml',         
         's3_fg0.01': 'security3_e2e_faster_rcnn_R-101-FPN_fg0.01_1x.yaml',                       
         's3_fg0.1': 'security3_e2e_faster_rcnn_R-101-FPN_fg0.1_1x.yaml',
         's3_bgl0.1': 'security3_e2e_faster_rcnn_R-101-FPN_bgl0.1_1x.yaml',                  
	     's4': 'security4_e2e_faster_rcnn_R-101-FPN_1x.yaml',
	     's7': 'security7_e2e_faster_rcnn_R-101-FPN_1x.yaml',
	     's3_ms': 'security3_e2e_faster_rcnn_R-101-FPN_ms_1x.yaml',
	     's4_ms': 'security3_e2e_faster_rcnn_R-101-FPN_ms_1x.yaml',
	     'r1': 'security1_retinanet_R-101-FPN_1x.yaml',
	     'r1a': 'security1a_retinanet_R-101-FPN_1x.yaml',		 		 		 
	     'r2': 'security2_retinanet_R-101-FPN_1x.yaml',
	     'r2p': 'security2_retinanet_R-101-FPN_1x.yaml',		 
	     'r3': 'security3_retinanet_R-101-FPN_1x.yaml',
	     'r4': 'security4_retinanet_R-101-FPN_1x.yaml'}

    if model not in model_map.keys():
        print 'None-Supported model %s' % model
        raise

    return model_map[model]
    
# plot_roc
if __name__ == '__main__':
    if len(sys.argv) < 2:
        print "Usage:"
        print '\tpython %s model' % (__file__)
        print '\t-- model: security7 security4 security4_ms security3 or security3_ms'
        sys.exit(0)

output_root = '/home/amax/workspace/Detectron/detectron-output'
val_data = 'security_train_real_package_val'
model_name =  get_res_folder(sys.argv[1])
gt_data = load_json_file('/home/amax/data/securityData/annotations/instances_train_Real-Package_val.json')
if sys.argv[1][0] == 's':
    val_res_file = os.path.join(output_root, model_name, 'test/test/%s/generalized_rcnn/bbox_%s_results.json' % (val_data, val_data))
    if os.path.exists(val_res_file):
        test_res_file = os.path.join(output_root, model_name, 'test/test/%s/generalized_rcnn/bbox_%s_results.json' % (test_data, test_data))
    else:
        val_res_file = os.path.join(output_root, model_name, 'test/%s/generalized_rcnn/bbox_%s_results.json' % (val_data, val_data))    
        test_res_file = os.path.join(output_root, model_name, 'test/%s/generalized_rcnn/bbox_%s_results.json' % (test_data, test_data))
        
elif sys.argv[1][0] == 'r':
    val_res_file = os.path.join(output_root, model_name, 'test/%s/retinanet/bbox_%s_results.json' % (val_data, val_data))
    test_res_file = os.path.join(output_root, model_name, 'test/%s/retinanet/bbox_%s_results.json' % (test_data, test_data))
else:
    print 'None-Supported model %s' % sys.argv[1]
    raise
    
det_data = load_json_file(val_res_file)
det_data = group_bboxs(det_data)
gt_data = group_bboxs(gt_data['annotations'])
judged_det = judge_detection_results(det_data, gt_data)

categories = ['knife', 'gun']
rocs = generate_rocs(judged_det, categories)
false_rates = [0.01, 0.03, 0.05, 0.1]
heads = ['False Rate', 'Overall'] + categories

print '-------------------------------------------'
print '%s' % '\t'.join(heads)

for fr in false_rates:
    line = ['%d'%fr]
    for (ci, roc) in enumerate(rocs):
        tmp = get_threshold_by_false_rate(roc, fr)
        line.append(tmp[0])
    print '%s' % '\t'.join(line)
print '-------------------------------------------'
        
# roc = generate_roc(judged_det, judged_det2)
# with open(os.path.join(output_root, model_name, 'pn_curve.txt'), 'wt') as fp:
#     for line in roc:
#         fp.write('%f %f %f\n' % line)
# print 'saved pn_curve.'

# print '##########Thres by false rate #############'
# print get_threshold_by_false_rate(roc, 0.01)
# print get_threshold_by_false_rate(roc, 0.0312)
# print get_threshold_by_false_rate(roc, 0.059)
# print get_threshold_by_false_rate(roc, 0.074)
# print get_threshold_by_false_rate(roc, 0.0913)
# print '###########Thres by hit rate  #############'
# print get_threshold_by_hit_rate(roc, 0.8)
# print get_threshold_by_hit_rate(roc, 0.9)
# print get_threshold_by_hit_rate(roc, 0.95)
# print get_threshold_by_hit_rate(roc, 0.98)
# print get_threshold_by_hit_rate(roc, 0.99)
